In [18]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics.utils.plotting import plot_labels
import pandas as pd
from sklearn.metrics import confusion_matrix
import torch
import os
import yaml
from datetime import datetime

def setup_eval_directory():
    """Create directory for saving evaluation results"""
    eval_dir = os.path.join('/kaggle/working', 'eval_res')
    os.makedirs(eval_dir, exist_ok=True)
    
    # Create timestamp subdirectory to avoid overwriting
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    result_dir = os.path.join(eval_dir, timestamp)
    os.makedirs(result_dir, exist_ok=True)
    
    return result_dir

def create_dataset_yaml():
    """Recreate the dataset YAML file with the same configuration used during training."""
    dataset_config = {
        'path': '/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data',
        'train': '/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/train/images',
        'val': '/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/images',
        'test': '/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/test/images',
        'names': {
            0: 'Hardhat',
            1: 'Mask',
            2: 'NO-Hardhat',
            3: 'NO-Mask',
            4: 'NO-Safety Vest',
            5: 'Person',
            6: 'Safety Cone',
            7: 'Safety Vest',
            8: 'machinery',
            9: 'vehicle'
        }
    }
    
    yaml_path = '/kaggle/working/dataset.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(dataset_config, f, default_flow_style=False)
    return yaml_path

def plot_confusion_matrix(model, save_dir, conf_thresh=0.25):
    """
    Plot and save confusion matrix for the model predictions.
    """
    try:
        # Get validation predictions
        val_results = model.val(conf=conf_thresh, data='/kaggle/working/dataset.yaml')
        
        # Extract true and predicted labels
        true_labels = []
        pred_labels = []
        
        for i, batch in enumerate(val_results.boxes):
            true_labels.extend(batch.cls.cpu().numpy())
            pred_labels.extend(batch.pred_cls.cpu().numpy())
        
        # Create confusion matrix
        classes = model.names
        cm = confusion_matrix(true_labels, pred_labels)
        
        # Plot confusion matrix
        plt.figure(figsize=(12, 10))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=classes.values(),
                    yticklabels=classes.values())
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        # Save plot
        save_path = os.path.join(save_dir, 'confusion_matrix.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved confusion matrix to {save_path}")
        
    except Exception as e:
        print(f"Error in plotting confusion matrix: {str(e)}")

def plot_pr_curve(model, save_dir):
    """
    Plot and save Precision-Recall curve for each class.
    """
    try:
        results = model.val(data='/kaggle/working/dataset.yaml')
        
        plt.figure(figsize=(12, 8))
        for c in range(len(model.names)):
            if c in results.prec_recall:
                precision, recall = results.prec_recall[c]
                plt.plot(recall, precision, label=f'{model.names[c]}')
        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves by Class')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()
        
        # Save plot
        save_path = os.path.join(save_dir, 'pr_curves.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved PR curves to {save_path}")
        
    except Exception as e:
        print(f"Error in plotting PR curve: {str(e)}")

def plot_f1_curve(model, save_dir):
    """
    Plot and save F1-score vs confidence threshold.
    """
    try:
        plt.figure(figsize=(10, 6))
        conf_thresholds = np.linspace(0, 1, 20)
        f1_scores = []
        
        for conf in conf_thresholds:
            val_results = model.val(conf=conf, data='/kaggle/working/dataset.yaml')
            f1_scores.append(val_results.box.f1)
        
        plt.plot(conf_thresholds, f1_scores)
        plt.xlabel('Confidence Threshold')
        plt.ylabel('F1 Score')
        plt.title('F1 Score vs Confidence Threshold')
        plt.grid(True)
        
        # Save plot
        save_path = os.path.join(save_dir, 'f1_curve.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved F1 curve to {save_path}")
        
    except Exception as e:
        print(f"Error in plotting F1 curve: {str(e)}")

def plot_training_curves(model, save_dir):
    """
    Plot and save training metrics.
    """
    try:
        # Try to get the results from the runs directory
        results_file = os.path.join('/kaggle/working/runs/detect/ppe_detection/results.csv')
        if os.path.exists(results_file):
            results = pd.read_csv(results_file)
        else:
            print("Training history not found. Skipping training curves.")
            return
        
        metrics_to_plot = {
            'box_loss': 'Box Loss',
            'cls_loss': 'Classification Loss',
            'dfl_loss': 'DFL Loss',
            'map50': 'mAP@50',
            'map75': 'mAP@75'
        }
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.ravel()
        
        for idx, (metric, title) in enumerate(metrics_to_plot.items()):
            if metric in results.columns:
                axes[idx].plot(results.index, results[metric])
                axes[idx].set_title(title)
                axes[idx].set_xlabel('Epoch')
                axes[idx].set_ylabel('Value')
                axes[idx].grid(True)
        
        plt.tight_layout()
        
        # Save plot
        save_path = os.path.join(save_dir, 'training_curves.png')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved training curves to {save_path}")
        
    except Exception as e:
        print(f"Error in plotting training curves: {str(e)}")

def save_metrics_summary(metrics, save_dir):
    """Save metrics summary to a text file"""
    summary_path = os.path.join(save_dir, 'metrics_summary.txt')
    with open(summary_path, 'w') as f:
        f.write("Model Performance Metrics:\n")
        f.write(f"mAP@50: {metrics.box.map50:.3f}\n")
        f.write(f"mAP@75: {metrics.box.map75:.3f}\n")
        f.write(f"mAP@50-95: {metrics.box.map:.3f}\n")
    print(f"Saved metrics summary to {summary_path}")

def evaluate_model(model_path):
    """
    Perform comprehensive model evaluation and save all results.
    """
    try:
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found at: {model_path}")
        
        # Create evaluation directory
        save_dir = setup_eval_directory()
        print(f"Saving evaluation results to: {save_dir}")
        
        # Create dataset.yaml
        yaml_path = create_dataset_yaml()
        print(f"Created dataset config at: {yaml_path}")
        
        # Load model
        model = YOLO(model_path)
        
        print("Generating and saving evaluation plots...")
        
        # Generate and save all plots
        plot_confusion_matrix(model, save_dir)
        plot_pr_curve(model, save_dir)
        plot_f1_curve(model, save_dir)
        plot_training_curves(model, save_dir)
        
        # Get and save metrics
        metrics = model.val(data=yaml_path)
        save_metrics_summary(metrics, save_dir)
        
        print("\nEvaluation complete! All results saved to:", save_dir)
        
    except Exception as e:
        print(f"Error in model evaluation: {str(e)}")

# Example usage
if __name__ == "__main__":
    model_path = "/kaggle/input/ppe_detection_model/other/default/1/ppe_detection_model.pt"  # Update with your model path
    evaluate_model(model_path)

Saving evaluation results to: /kaggle/working/eval_res/20241125_052403
Created dataset config at: /kaggle/working/dataset.yaml
Generating and saving evaluation plots...
Ultralytics 8.3.36 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/labels... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<00:00, 716.79it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.16it/s]


                   all        114        697       0.88      0.726      0.832      0.563
               Hardhat         42         79      0.903      0.785      0.884      0.629
                  Mask         19         21      0.906      0.857      0.898      0.703
            NO-Hardhat         37         69      0.873        0.6      0.758      0.458
               NO-Mask         44         74      0.893      0.566      0.771      0.433
        NO-Safety Vest         56        106      0.881        0.7      0.817      0.534
                Person         84        166      0.885      0.742      0.856      0.597
           Safety Cone         13         44      0.871      0.841       0.89      0.535
           Safety Vest         28         41      0.934      0.805       0.91      0.651
             machinery         26         55       0.96       0.87       0.92      0.696
               vehicle         16         42      0.695      0.488      0.614      0.389
Speed: 2.6ms preproce

...

Speed: 6.3ms preprocess, 3.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val41
Error in plotting F1 curve: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (20,) + inhomogeneous part.
Training history not found. Skipping training curves.
Ultralytics 8.3.36 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


val: Scanning /kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/labels... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<00:00, 785.67it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.88it/s]


                   all        114        697       0.88      0.726      0.798      0.507
               Hardhat         42         79      0.903      0.785      0.874      0.581
                  Mask         19         21      0.906      0.857      0.867      0.664
            NO-Hardhat         37         69      0.873        0.6      0.703      0.393
               NO-Mask         44         74      0.893      0.566      0.683      0.353
        NO-Safety Vest         56        106      0.881        0.7      0.763      0.462
                Person         84        166      0.885      0.742      0.843      0.551
           Safety Cone         13         44      0.871      0.841      0.867      0.478
           Safety Vest         28         41      0.934      0.805      0.916      0.611
             machinery         26         55       0.96       0.87      0.921      0.658
               vehicle         16         42      0.695      0.488      0.548      0.322
Speed: 16.0ms preproc